In [75]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import yaml
import pathlib

In [76]:
curr_dir = pathlib.Path().resolve()
print(curr_dir)

/home/inductive-anks/yield-prediction/Yield-Prediction/Code Files/Subdilstrict-Shape-Files


In [77]:
home_dir = curr_dir.parent.parent

In [78]:
df_subdist_shape = gpd.read_file(home_dir.as_posix() + '/data/All-India-Subdistrict-Shape-File/SUBDISTRICT_BOUNDARY.shp')

In [79]:
df_village_list = pd.read_csv(home_dir.as_posix() + '/data/Tehsil Data/Tehsil_Data_With_Single_Village.csv')

In [80]:
df_SOI_shape = gpd.read_file(home_dir.as_posix() + '/data/All-Inida-SOI/All-India-Villages-SOI.shp')

In [81]:
df_SOI_shape.shape

(599742, 5)

In [82]:
df_SOI_shape.head()

,NAME,dtnmcoc,state,tehnmcoc,geometry
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101..."
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775..."
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418..."
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894..."
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541..."


In [83]:
df_SOI_shape.rename(columns={'NAME': 'SOI_Village_Name', 'dtnmcoc': 'SOI_District_Name', 'tehnmcoc':'SOI_Tehsil_Name'}, inplace=True)

In [84]:
df_SOI_shape.head()

,SOI_Village_Name,SOI_District_Name,state,SOI_Tehsil_Name,geometry
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101..."
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775..."
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418..."
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894..."
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541..."


In [85]:
df_village_list.head()

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
0,2008-2011,andhra pradesh,ranga reddy,iv,puduru,40,pedhaumethala
1,2008-2011,andhra pradesh,medak,v,sadasivpet,49,nandi kandi
2,2008-2011,andhra pradesh,medak,v,zaiheerabad,50,huggelly
3,2008-2011,andhra pradesh,srikakulam,i,etheria,1,ponnada
4,2008-2011,andhra pradesh,srikakulam,i,tekkali,2,tirlangi


In [86]:
df_subdist_shape.head()

,District,STATE,TEHSIL,Shape_Leng,Shape_Area,geometry
0,BIL>SPUR,HIMACHAL PRADESH,BHARARI,50253.228763,9.000306e+07,"POLYGON ((3690806.89 4831388.917, 3690871.421 ..."
1,BIL>SPUR,HIMACHAL PRADESH,BIL>SPUR,93446.685818,1.703525e+08,"POLYGON ((3699861.726 4812292.061, 3699965.144..."
2,BIL>SPUR,HIMACHAL PRADESH,GHURMW|N,105834.665205,1.906542e+08,"POLYGON ((3692609.535 4827764.245, 3692614.853..."
3,BIL>SPUR,HIMACHAL PRADESH,JHAND@TA,112831.475031,3.205376e+08,"POLYGON ((3667564.424 4819909.775, 3667603.174..."
4,BIL>SPUR,HIMACHAL PRADESH,NAMHOL,61752.616994,1.034979e+08,"POLYGON ((3701617.644 4803943.439, 3701666.832..."


## Converting to Lower Case

In [87]:
df_subdist_shape = df_subdist_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_1639/228283043.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_subdist_shape = df_subdist_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [88]:
df_village_list = df_village_list.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_1639/990218482.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_village_list = df_village_list.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [89]:
df_SOI_shape = df_SOI_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_1639/3172920881.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_SOI_shape = df_SOI_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)


## Test

In [90]:
village_list_SOI_mapping = gpd.read_file(home_dir.as_posix() + '/data/Village_List-SOI-Mapping(Main)/Village_List-SOI-Mapping(Main).shp')

In [91]:
village_list_SOI_mapping.shape

(3278, 11)

In [92]:
village_list_SOI_mapping.head()

,STATE,YEAR,ZONE,DISTRICT,slnococ,TEHSIL,VL,sdnm_11,SOI,Similarity,geometry
0,punjab,2008-11,1,gurdaspur,2,batala,ammo nagal,batala,ammo nagal,1.0,"POLYGON ((75.24774 31.75, 75.24879 31.75118, 7..."
1,punjab,2008-11,1,amrtisar,3,ajnala,bhalapind,ajnala,bhalapind,1.0,"POLYGON ((74.78681 31.79373, 74.78854 31.79339..."
2,punjab,2008-11,1,amritsar,4,amritsar-i,ballian manjpur,amritsar i,ballian manjpur,1.0,"POLYGON ((75.08233 31.59099, 75.08259 31.59082..."
3,punjab,2008-11,1,tarntarn,5,patti,kirtowal,patti,kirtowal,1.0,"POLYGON ((74.90056 31.21235, 74.90026 31.2118,..."
4,punjab,2008-11,1,hoshiarpur,6,mukerian,gera,mukerian,gera,1.0,"POLYGON ((75.79117 31.98615, 75.79183 31.98424..."


In [93]:
df_village_list.head()

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
0,2008-2011,andhra pradesh,ranga reddy,iv,puduru,40,pedhaumethala
1,2008-2011,andhra pradesh,medak,v,sadasivpet,49,nandi kandi
2,2008-2011,andhra pradesh,medak,v,zaiheerabad,50,huggelly
3,2008-2011,andhra pradesh,srikakulam,i,etheria,1,ponnada
4,2008-2011,andhra pradesh,srikakulam,i,tekkali,2,tirlangi


In [94]:
df_village_list = df_village_list.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/tmp/ipykernel_1639/3965484942.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_village_list = df_village_list.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [95]:
import pandas as pd

filtered_rows = []

for idx, row in village_list_SOI_mapping.iterrows():
    village = row['VL']
    
    match = df_village_list[
        (df_village_list['Village Name'] == village)
    ]
    
    if not match.empty:
        filtered_rows.append(row)

filtered_df = pd.DataFrame(filtered_rows)

In [96]:
filtered_df.shape

(2533, 11)

In [97]:
df_village_list.shape

(2540, 7)

In [98]:
import pandas as pd

remaining_rows = df_village_list[
    ~df_village_list.apply(lambda x: 
        ((village_list_SOI_mapping['TEHSIL'] == x['Tehsil Name'])).any(), axis=1)
]

In [99]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [100]:
remaining_rows.shape

(63, 7)

In [101]:
remaining_rows.to_csv('/home/inductive-anks/yield-prediction/Yield-Prediction/data/Tehsil Data/remaining_unmatched_villages.csv', index=False)

In [102]:
remaining_rows.sample(8)

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
660,2011-2014,andhra pradesh,ananthapur,zone-iii,kothacheruvu,32,bandapalli
585,2008-2011,karnataka,bhalki,01,bidar,2.0,bhatramba
600,2008-2011,karnataka,raibhag,03,belgaum,17.0,kudachi
959,2011-2014,rajasthan,s. ganganagar,iii,anoopgarh,17,79gb/80gb/88gb
92,2008-2011,chhattisgarh,raipur,ii,gariyaband,10,malgaon
620,2008-2011,karnataka,hunsur,07,mysore,37.0,devanahalli
595,2008-2011,karnataka,soundatti,03,belgaum,12.0,karikatti
359,2008-2011,orissa,rayagada,ii,kasipur,22,"tikiri,jamuguda"
